In [1]:
# import necessary dependecies 
import re # regular expression to check if theres a pattern 
import tiktoken # Implementing Byte Pair Encoding using exisitng open-source library 

import torch 
import torch.nn as nn 
from torch.utils.data import Dataset, DataLoader 

In [2]:
# Check if Mac supports PyTorch Acceleration with Apple Silicon Chip 
print(torch.backends.mps.is_available())

True


In [3]:
class SimpleTokenizer: 
    def __init__(self, vocab):
        self.string_to_integer = vocab
        self.integer_to_string = {i:s for s,i in vocab.items()} 
        
    def encode(self, text):
        preprocessed = re.split(r'([,.?_!"()\']|--|\s)', text)  # Split text by specific punctuation and whitespace
        preprocessed = [item.strip() for item in preprocessed if item.strip()]  # Strip and filter out empty items
        preprocessed = [item if item in self.string_to_integer else "<|unk|>" for item in preprocessed]

        ids = [self.string_to_integer[s] for s in preprocessed if s in self.string_to_integer]  # Map the text to ids
        return ids

    def decode(self, ids): 
        text = " ".join([self.integer_to_string[i] for i in ids])
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text
    

In [4]:
class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride): 
        self.input_ids = []
        self.target_ids = [] 
        
        token_ids = tokenizer.encode(txt, allowed_special = {"|endoftext|>"}) 
        
        for i in range(0, len(token_ids) - max_length, stride): # Creating context window 
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk)) # Creates a input matricies  
            self.target_ids.append(torch.tensor(target_chunk)) # Creates a input matricies  
            
    def __len__(self): 
        return len(self.input_ids)
    
    def __getitem__(self, idx): 
        return self.input_ids[idx], self.target_ids[idx]


In [5]:
def create_dataloader_v1(txt, batch_size =4, max_length=256, stride=128, shuffle=True, drop_last=True, num_worker = 0):
    tokenizer = tiktoken.get_encoding("gpt2") 
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    
    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=shuffle,
        drop_last=drop_last, 
        num_workers=num_worker)
    
    return dataloader

In [6]:
with open("The_Verdict.txt", "r", encoding="utf=8") as f: 
    text = f.read() 
    
result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
preprocessed = [item for item in result if item.strip()]                #further remove whitespaces from string list 
all_tokens = sorted(list(set(preprocessed)))               # set() builds a hash set, and list() makes the set into a list. 
all_tokens.extend(["<|endoftext|>", "<|unk|>"])

vocab = {token:integer for integer, token in enumerate(all_tokens)}

vocab


{'!': 0,
 '"': 1,
 "'": 2,
 '(': 3,
 ')': 4,
 '*': 5,
 ',': 6,
 '--': 7,
 '.': 8,
 '0': 9,
 '1': 10,
 '16': 11,
 '1908': 12,
 '1929': 13,
 '2024': 14,
 '4': 15,
 ':': 16,
 ';': 17,
 '?': 18,
 'A': 19,
 'About': 20,
 'Ah': 21,
 'Among': 22,
 'And': 23,
 'Are': 24,
 'Arrt': 25,
 'As': 26,
 'At': 27,
 'Attribution-ShareAlike': 28,
 'Be': 29,
 'Begin': 30,
 'Burlington': 31,
 'But': 32,
 'By': 33,
 'Carlo': 34,
 'Chicago': 35,
 'Claude': 36,
 'Come': 37,
 'Commons': 38,
 'Creative': 39,
 'Croft': 40,
 'Destroyed': 41,
 'Devonshire': 42,
 'Don': 43,
 'Dubarry': 44,
 'During': 45,
 'Edith': 46,
 'Emperors': 47,
 'Exported': 48,
 'FDL': 49,
 'Florence': 50,
 'For': 51,
 'GNU': 52,
 'Gallery': 53,
 'Gideon': 54,
 'Gisburn': 55,
 'Gisburns': 56,
 'Grafton': 57,
 'Greek': 58,
 'Grindle': 59,
 'Grindles': 60,
 'HAD': 61,
 'Had': 62,
 'Hang': 63,
 'Has': 64,
 'He': 65,
 'Her': 66,
 'Hermia': 67,
 'His': 68,
 'How': 69,
 'I': 70,
 'If': 71,
 'In': 72,
 'It': 73,
 'Jack': 74,
 'January': 75,
 'Jove'

In [7]:
# Test - Instantiate an object 
tokenizer = SimpleTokenizer(vocab) 
ids = tokenizer.encode(text) 

ecoded_text = tokenizer.decode(ids) 


In [8]:
ids

[113,
 127,
 46,
 132,
 12,
 48,
 532,
 136,
 794,
 105,
 11,
 6,
 14,
 70,
 61,
 177,
 1084,
 74,
 55,
 893,
 142,
 289,
 541,
 7,
 1083,
 142,
 555,
 488,
 443,
 7,
 986,
 641,
 1162,
 774,
 563,
 1040,
 1097,
 724,
 1097,
 590,
 1068,
 6,
 623,
 1069,
 593,
 788,
 604,
 551,
 6,
 588,
 569,
 418,
 604,
 817,
 6,
 722,
 142,
 918,
 1188,
 6,
 185,
 449,
 602,
 623,
 142,
 1150,
 794,
 1069,
 102,
 8,
 3,
 119,
 70,
 893,
 1084,
 641,
 1207,
 585,
 238,
 103,
 802,
 50,
 8,
 4,
 1,
 113,
 593,
 788,
 604,
 551,
 1,
 7,
 1068,
 1162,
 1175,
 1069,
 1198,
 275,
 641,
 8,
 70,
 277,
 590,
 85,
 8,
 54,
 120,
 7,
 604,
 658,
 35,
 975,
 7,
 370,
 604,
 1124,
 143,
 8,
 1,
 91,
 341,
 641,
 2,
 927,
 553,
 1097,
 943,
 1069,
 1143,
 788,
 760,
 838,
 2,
 1168,
 1218,
 272,
 70,
 407,
 2,
 1049,
 1079,
 788,
 1068,
 6,
 84,
 8,
 101,
 7,
 1069,
 706,
 1097,
 25,
 640,
 173,
 70,
 1079,
 788,
 8,
 1,
 113,
 1202,
 6,
 794,
 85,
 8,
 120,
 2,
 927,
 692,
 6,
 756,
 642,
 141,
 924,
 141,
 207

In [9]:
decoded_text

NameError: name 'decoded_text' is not defined

In [171]:
for i, item in enumerate(list(vocab.items())[-5:]):
    print(item)

('younger', 1214)
('your', 1215)
('yourself', 1216)
('<|endoftext|>', 1217)
('<|unk|>', 1218)


In [172]:
# Test end of text token in the case of additional test source
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1, text2))
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [173]:
tokenizer = SimpleTokenizer(vocab)
print(tokenizer.encode(text))


[1218, 6, 402, 1213, 687, 1054, 18, 1217, 72, 1069, 1035, 1065, 788, 1069, 1218, 8]


In [174]:
print(tokenizer.decode(tokenizer.encode(text)))

<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.


# TEST - Byte Pair Encoding 

In [175]:
tokenizer = tiktoken.get_encoding("gpt2") 
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 262, 20562, 13]


In [176]:
print(type(integers))

<class 'list'>


In [177]:
text = "AKwirw ier"
integers = tokenizer.encode(text)
tokenizer.decode(integers)

'AKwirw ier'

In [178]:
with open("The_Verdict.txt", 'r', encoding="utf=8") as f:
    raw_text = f.read()
    
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
first_batch

 

[tensor([[  464,  4643, 11600,   628]]),
 tensor([[ 4643, 11600,   628,   198]])]

In [179]:
second_batch = next(data_iter)

In [180]:
second_batch # Each stride is a representation of the next context window size

[tensor([[ 4643, 11600,   628,   198]]),
 tensor([[11600,   628,   198,   197]])]

In [181]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter) 
print("Inputs:\n", inputs) 
print("\nTargets:\n", targets)

Inputs:
 tensor([[  464,  4643, 11600,   628],
        [  198,   197,   197,   197],
        [  197,   197,  7407,   342],
        [  854, 41328,   628,   628],
        [  198,   198,  1129,  2919],
        [  628,   628,   198,   198],
        [ 3109,  9213,   422, 11145],
        [  271,  1668,   319,  2693]])

Targets:
 tensor([[ 4643, 11600,   628,   198],
        [  197,   197,   197,   197],
        [  197,  7407,   342,   854],
        [41328,   628,   628,   198],
        [  198,  1129,  2919,   628],
        [  628,   198,   198,  3109],
        [ 9213,   422, 11145,   271],
        [ 1668,   319,  2693,  1467]])


In [182]:
# Initializing embedded vectors, the input variables where the weights later gets applied to

inputs_ids = torch.tensor([2, 3, 5, 1])

In [183]:
vocab_size = 6 
output_dim = 3 
 
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight) 

Parameter containing:
tensor([[ 1.2753, -0.2010,  0.9624],
        [ 0.2492, -0.4845, -2.0929],
        [-0.8199, -0.4210, -0.9620],
        [ 1.2825, -0.3430, -0.6821],
        [-0.9887, -1.7018, -0.7498],
        [-1.1285,  0.4135,  0.2892]], requires_grad=True)


In [184]:
print(embedding_layer(torch.tensor([3])))

tensor([[ 1.2825, -0.3430, -0.6821]], grad_fn=<EmbeddingBackward0>)


In [185]:
vocab_size = 50257
output_dim = 256

token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [186]:
max_length = 4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle = True)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs:\n", inputs)
print("\nInputs shape:\n", inputs.shape)

Token IDs:
 tensor([[  922,    12, 49705,    11],
        [  438,  4360,   612,   338],
        [  612,  1997,   319,  4534],
        [  407,  2957,   683,   736],
        [16957,  1696,   414,     1],
        [11978,  2119,    11,   351],
        [ 2060,   530,   287,   262],
        [   11,   262, 15910,   286]])

Inputs shape:
 torch.Size([8, 4])


In [187]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)




torch.Size([8, 4, 256])


In [188]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [189]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


In [190]:

input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)


torch.Size([8, 4, 256])


In [191]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     (x^4)
    [0.77, 0.25, 0.10], # one      (x^5)
    [0.05, 0.80, 0.55]  # step     (x^6)
    ])

inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.8700, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [192]:
# Calculate attention scores 
# This process effectively takes the dot product of each embedded query token and takes the dot product including itself to compute a attention score
# An attention score is to guage the similarity between the current embedded vector with respect to all the embedded vector including itself.


# Here we are calculating the intermediate attention scores for one of the query token 

query = inputs[1] # Taking the embedded vector of index 1 as the query token 

attn_score_2 = torch.empty(inputs.shape[0]) # Create a data structure to hold the attention score of the embedded vector with respect to each input vector in this iteration

for i, x_i in enumerate(inputs):
    attn_score_2[i] = torch.dot(x_i, query) 
 
attn_score_2 # attention score of the query embedded vector with respect to each input embedded vector

# Each scalar product is a representation of how similar or aligned between the query and input embedded vector



tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])

In [193]:
print("Attention Score: ", attn_score_2)

Attention Score:  tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [194]:
# Attention weights are derived by normalizing the attention scores
# all the attention weights should sum up to one

attn_weights_2_tmp = attn_score_2 / attn_score_2.sum() 

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

# This normalization technique is not the most optimal. Only used for interpretiability. 

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


In [195]:
# Usually normalization is achieved through softmax

def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)
attn_weights_2_naive = softmax_naive(attn_score_2)
print("Attention weights:", attn_weights_2_naive)
 
print("Sum:", attn_weights_2_naive.sum())


Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [196]:
# Its also better to use the softmax function since it prevents under and overflow. Pytorch highly optimized it.

attn_weights_2 = torch.softmax(attn_score_2, dim=0)
print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


In [197]:
atten_scores = torch.empty(6,6)
for i, x_j in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        atten_scores[i, j] = torch.dot(x_i, x_j)
        
        
atten_scores

tensor([[0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [198]:
# The code above can be written use @ which is faster for matrix multiplication

atten_scores = inputs @ inputs.T
atten_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [199]:
atten_weights = torch.softmax(atten_scores, dim=1)
atten_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [200]:
# Double check that rows sum up to one 
sum_2_row = sum([0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452])
sum_2_row

0.9999

In [201]:
# Check that all rows sum to 1 

atten_weights.sum(dim = 1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [202]:
all_context_vecs = atten_weights @ inputs
all_context_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

# Re-Creating Self Attention

In [203]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2


In [204]:
inputs, x_2

(tensor([[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]),
 tensor([0.5500, 0.8700, 0.6600]))

In [205]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)


W_query.shape, W_query


(torch.Size([3, 2]),
 Parameter containing:
 tensor([[0.2961, 0.5166],
         [0.2517, 0.6886],
         [0.0740, 0.8665]]))

In [206]:
# After creating weight matricies you multiply them each into copies of input vectors to create q,k,v vectors

query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key 
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


In [207]:

keys = inputs @ W_key 
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


In [208]:
# The dot product of the key and query vector together is the the attention score
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


In [209]:
# Since we have 6 inputs embedded vector againt 1 embedded vector there would need to be 6 attention scores
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)


tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [210]:
# After computing the attention score we find the attention weight by normalizing the values with softmax

# The difference to earlier is that we now scale the attention scores by dividing them by the square root of the embedding dimension, 
#  (i.e., d_k**0.5):

d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [211]:
# Now we compute the context vector by multiplying each value vector within the context window with the current respective weight and then summming them to obtain the context vector
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [212]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

d_in, d_out

(3, 2)

In [213]:

import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [17]:
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs)) 



NameError: name 'inputs' is not defined

In [10]:

# Reuse the query and key weight matrices of the
# SelfAttention_v2 object from the previous section for convenience
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

NameError: name 'sa_v2' is not defined

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert d_out % num_heads == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads  # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x)  # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec)  # optional projection

        return context_vec